In [1]:
import cv2
import numpy as np
from statistics import mean, median, mode

In [2]:
def calculateSpatialInfo(img: np.ndarray) -> float:
    """
    Calculate spatial information from a grayscale image using Sobel filters.

    Args:
        img: A grayscale image as a numpy array.

    Returns:
        The spatial information as a float.
    """
    sh = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=1)
    sv = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=1)

    sobel_image = np.sqrt(np.square(sh) + np.square(sv))

    si_mean = np.sum(sobel_image) / (sobel_image.shape[0] * sobel_image.shape[1])
    si_rms = np.sqrt(np.sum(sobel_image ** 2) / (sobel_image.shape[0] * sobel_image.shape[1]))
    si_stdev = np.sqrt(np.sum(sobel_image ** 2 - si_mean ** 2) / (sobel_image.shape[0] * sobel_image.shape[1]))

    return si_stdev

In [3]:
def getNeighbour(image: np.ndarray, i: int, j: int) -> list:
    neighbour = [[], [], []]
    shape = image.shape
    for di, dj in [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]:
        curr_i = i + di
        curr_j = j + dj
        if curr_i < 0 or curr_i >= shape[0]:
            continue
        if curr_j < 0 or curr_j >= shape[1]:
            continue
        for color in [0, 1, 2]:
            neighbour[color].append(image[curr_i][curr_j][color])
    return neighbour


In [4]:
def polishingImage(resized: np.ndarray, func: callable) -> np.ndarray:
    shape = resized.shape
    polished = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            neighbour = getNeighbour(resized, i, j)
            polished[i][j] = [func(neighbour[0]), func(neighbour[1]), func(neighbour[2])]
    return polished

In [5]:
def resizeImage(original: np.ndarray, x_rate: float, y_rate: float, func: callable = None) -> np.ndarray:
    shape = list(original.shape)
    shape[0] = int(shape[0]*x_rate)
    shape[1] = int(shape[1]*y_rate)
    resized = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            orig_i, orig_j = int(i/x_rate),int(j/y_rate)
            if func:
                neighbour = getNeighbour(original, orig_i, orig_j)
                resized[i][j] = [func(neighbour[0]), func(neighbour[1]), func(neighbour[2])]
            else:
                resized[i][j] = [*original[orig_i][orig_j]]
            
    return resized


In [ ]:
original_images = ['a', 'b', 'c']

In [11]:
# Calculate images spatial info
images = {}
for letter in original_images:
    name = letter+'_gray'
    images[name] = cv2.imread(f'images/{letter}.jpg',cv2.IMREAD_GRAYSCALE)
    print(f'Spatial info of {letter}: {calculateSpatialInfo(images[name])}')

Spatial Info of a: 20.65579411743317
Spatial Info of b: 23.20730158646722
Spatial Info of c: 45.468247897791535


In [12]:
for letter in original_images:
    name = f'{letter}_resized_polish'
    images[letter] = cv2.imread(f'images/{letter}.jpg')
    images[name] = resizeImage(images[letter], 2, 2, mean)
    cv2.imwrite(f'images/{name}.jpg', images[name])

In [9]:
for letter in original_images:
    name = f'{letter}_resized'
    pol_name = f'{letter}_polished'
    images[name] = resizeImage(images[letter], 2, 2)
    cv2.imwrite(f'images/{name}.jpg', images[name])
    images[pol_name] = polishingImage(images[name], mean)
    cv2.imwrite(f'images/{pol_name}.jpg', images[pol_name])


True